In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


In [2]:

def load_and_clean_csv(file_path):
    df = pd.read_csv(file_path, index_col=0)
    df.columns = df.columns.str.strip()
    df = df.apply(pd.to_numeric, errors='coerce')
    # strip the whitespace in the end of the index
    df.index = df.index.str.strip()
    # remove _1 in the end of the index
    df.index = df.index.str.replace("_1$", "", regex=True)
    # make 1.0 values NaN
    df = df.mask(df == 1.0)
    return df

def align_columns(df, columns_order):
    return df.reindex(columns=columns_order)

def plot_heatmap(df, title, mask=False):
    # ignore 1.0 values
    if mask:
        df = df.mask(df == 1.0)
    plt.figure(figsize=(10, 8))
    ax = sns.heatmap(df, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)        
    ax.set_title(title)
    ax.set_xlabel("Source model")
    ax.set_ylabel("Target model")
    plt.tight_layout() 
    return ax

def aggregate_transferability_scores(attacks):
    attack_scores = {name: attacks[name].mean().mean() for name in attacks}
    attack_scores_df = pd.DataFrame.from_dict(attack_scores, orient='index', columns=['Average Transferability'])
    return attack_scores_df.sort_values(by='Average Transferability', ascending=False)

def plot_transferability_scores(attack_scores_df, title):
    plt.figure(figsize=(20, 10))
    ax = sns.barplot(x=attack_scores_df.index, y='Average Transferability', data=attack_scores_df, palette='coolwarm', hue=attack_scores_df.index)
    ax.set_xlabel("Attack")
    ax.set_ylabel("Average Transferability")
    ax.set_title(title)
    plt.xticks(rotation=45)  # Rotate labels for better readability
    plt.tight_layout()
    return ax


def plot_3d_transferability(df, title):

    x_labels = df.columns
    y_labels = df.index
    x, y = np.meshgrid(range(len(x_labels)), range(len(y_labels)))
    z = df.to_numpy()

    _x = np.arange(len(x_labels))
    _y = np.arange(len(y_labels))
    _xx, _yy = np.meshgrid(_x, _y)
    x, y = _xx.ravel(), _yy.ravel()
    top = z.ravel()
    bottom = np.zeros_like(top)
    width = depth = 0.5

    fig = plt.figure(figsize=(10, 6))
    ax = fig.add_subplot(111, projection='3d')
    bars = ax.bar3d(x, y, bottom, width, depth, top, shade=True)

    # Setting axis labels
    ax.set_xlabel('Source Models')
    ax.set_ylabel('Target Models')
    ax.set_zlabel('Transferability Score')

    # Setting the tick labels with adjustments
    # Rotate x-axis labels for better readability
    ax.set_xticks(_x + width/2)
    ax.set_xticklabels(x_labels, rotation=45, ha='right', fontsize=10)

    # Display every other y-axis label to reduce crowding
    ax.set_yticks(_y[::2] + depth/2)
    ax.set_yticklabels(y_labels[::2], rotation=45, ha='right', fontsize=10)

    # Show the plot with adjustments
    plt.show()

def compare_model_types(df):
    possible_indices = df.index.unique()
    model_types = {
    "convolutional": ["resnet18", "resnet152", "densenet121", "densenet201", "convnext_tiny", "convnext_small"],
    "transformer": ["vit_l_32", "vit_b_16", "swin_v2_t", "swin_v2_s"]
    }
    
    indices_convolutional = [index for index in possible_indices if index in model_types["convolutional"]]
    indices_transformer = [index for index in possible_indices if index in model_types["transformer"]]
    
    transferability_from_convolutional_to_transformer = df.loc[indices_transformer, indices_convolutional].mean().mean()
    transferability_from_transformer_to_convolutional = df.loc[indices_convolutional, indices_transformer].mean().mean()
    
    print("Average transferability from convolutional to transformer models: ", transferability_from_convolutional_to_transformer)
    print("Average transferability from transformer to convolutional models: ", transferability_from_transformer_to_convolutional)
    
    return transferability_from_convolutional_to_transformer, transferability_from_transformer_to_convolutional
    
    
def filter_models(df, models):
    return df.loc[models, models]

def compare_model_complexity_and_transferability(df):
    clean_imagenet_acc = {
    'resnet18': 0.69758,
    'densenet121': 0.74434,
    'swin_v2_t': 0.82072,
    'convnext_tiny': 0.8252,
    }
    model_params = {
        'resnet18': 11689512,
    'densenet121': 7978856,
    'swin_v2_t': 28351570,
    'convnext_tiny': 28589128,
    }
    
    # source: https://pytorch.org/vision/main/models.html
    # Prepare data for plotting
    model_names = list(model_params.keys())
    params_values = [model_params[model] for model in model_names if model in df.index]
    transferability_values = [df.loc[model].values[0] for model in model_names]

    # Creating the scatter plot
    plt.figure(figsize=(10, 6))
    plt.scatter(params_values, transferability_values, color='blue')

    # Annotating the points
    for i, model in enumerate(model_names):
        plt.annotate(model, (params_values[i], transferability_values[i]), textcoords="offset points", xytext=(0,10), ha='center')

    plt.xlabel('Number of Parameters')
    plt.ylabel('Transferability Score')
    plt.title('Relationship between Model Size and Transferability Score')
    plt.xscale('log')  # Using a logarithmic scale for better visualization
    plt.grid(True)
    plt.show()
    
def transferability_stats(df):
    # ignore 1.0 values
    df_masked = df.mask(df == 1.0)
    
    print("Average transferability: ", df_masked.mean().mean())
    print("Standard deviation of transferability: ", df_masked.std().std())
    print("Minimum transferability: ", df_masked.min().min())
    print("Maximum transferability: ", df_masked.max().max())
    print("Median transferability: ", df_masked.median().median())
    
    

def main(attack_group="all", attack_name=None, models=None):
    # if attack name is specified, attack group is ignored

    # Load and clean the DataFrames
    attack_files = {
        "pgd_single": "./pgd_single/transferability.csv",
        "pgd_iterative": "./pgd_iterative/transferability.csv",
        "fgsm": "./fgsm/transferability.csv",
        "fgsm_targeted": "./fgsm_targeted/transferability.csv",
        "deepfool_iterative": "./deepfool_iterative/transferability.csv",
        "deepfool_single": "./deepfool_single/transferability.csv",
        "lots_iterative": "./lots_iterative/transferability.csv",
        "lots_single": "./lots_single/transferability.csv",
    }

    attacks = {name: load_and_clean_csv(path) for name, path in attack_files.items()}

    # Align the columns
    columns_order = attacks["deepfool_iterative"].columns

    for attack in attacks:
        attacks[attack] = align_columns(attacks[attack], columns_order)

    attack_groups = {
        "targeted": ["fgsm_targeted", "lots_iterative", "lots_single"],
        "untargeted": ["pgd_single", "pgd_iterative", "fgsm", "deepfool_iterative", "deepfool_single"],
        "iterative": ["lots_iterative", "deepfool_iterative", "pgd_iterative"],
        "single": ["lots_single", "pgd_single", "fgsm", "deepfool_single", "fgsm_targeted"],
        "all": ["lots_iterative", "deepfool_iterative", "pgd_iterative", "lots_single", "pgd_single", "fgsm", "deepfool_single", "fgsm_targeted"]
    }

    
    if attack_name:
        combined = attacks[attack_name]
    else:
        # Concatenate and group by index (attack names)
        dfs = [attacks[attack] for attack in attack_groups[attack_group]] 
        # also filter the attacks
        attacks = {name: attacks[name] for name in attack_groups[attack_group]}
        combined = pd.concat(dfs)

    

    # Clean index and group by mean
    combined.index = combined.index.str.strip()
    combined_average = combined.groupby(combined.index).mean()


    desired_order = [
        'resnet18', 'convnext_tiny', 'swin_v2_t', 'densenet121'
    ]
    combined_average = combined_average.reindex(desired_order)[desired_order]
    
    if models:
        combined_average = filter_models(combined_average, models)

    # Aggregate and plot transferability scores
    attack_scores_df = aggregate_transferability_scores(attacks)

    # Find and plot the most vulnerable models
    avg_transferability = combined_average.mean(axis=1)
    sorted_avg_transferability = avg_transferability.sort_values(ascending=False)
    sorted_avg_transf_model_df = pd.DataFrame(sorted_avg_transferability, columns=['Average Transferability'])
            

    return combined_average, attack_scores_df, sorted_avg_transf_model_df

def plot_tr_comparison(tr_2_cv, cv_2_tr):
    plt.figure(figsize=(10, 8))
    ax = sns.barplot(x=["Convolutional to Transformer", "Transformer to Convolutional"], y=[tr_2_cv, cv_2_tr], palette='coolwarm')
    ax.set_xlabel("Model Type")
    ax.set_ylabel("Average Transferability")
    ax.set_title("Average Transferability of Model Types")
    plt.tight_layout()
    return ax

if __name__ == "__main__":
    df, attack_scores_df , sorted_avg_transf_model_df  = main("all")
    #transferability_stats(df)
    #tr_2_cv, cv_2_tr = compare_model_types(df)
    #plot_tr_comparison(tr_2_cv, cv_2_tr)
    #plot_heatmap(df, "Cross-Model Average Transferability", mask=True)
    #compare_model_complexity_and_transferability(sorted_avg_transf_model_df)
    #plot_3d_transferability(df, "ImageNet")
    #plot_transferability_scores(attack_scores_df, "Average Transferability of Attacks")
    #plot_transferability_scores(sorted_avg_transf_model_df, "Average Transferability of Models (as targets)")
    plt.show()
    
sorted_avg_transf_model_df.loc["resnet18"].values[0]


0.5837231313889234

In [3]:
df

,resnet18,convnext_tiny,swin_v2_t,densenet121
resnet18,NaN,0.583134,0.577901,0.590134
convnext_tiny,NaN,NaN,NaN,NaN
swin_v2_t,NaN,NaN,NaN,NaN
densenet121,NaN,NaN,NaN,NaN
